<a href="https://colab.research.google.com/github/Sehn1302/Sehn1302/blob/main/Learn_assist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install deepface

In [4]:
# Create a sample user interactions log
log_data = """{'user_id': 1, 'task_id': 1, 'time_spent': 15, 'performance': 1, 'emotion': 'happy', 'timestamp': '2024-10-01 10:00:00'}
{'user_id': 1, 'task_id': 2, 'time_spent': 20, 'performance': 1, 'emotion': 'neutral', 'timestamp': '2024-10-01 10:15:00'}
{'user_id': 1, 'task_id': 3, 'time_spent': 10, 'performance': 0, 'emotion': 'sad', 'timestamp': '2024-10-01 10:30:00'}
{'user_id': 2, 'task_id': 1, 'time_spent': 30, 'performance': 1, 'emotion': 'happy', 'timestamp': '2024-10-01 11:00:00'}
{'user_id': 2, 'task_id': 2, 'time_spent': 25, 'performance': 0, 'emotion': 'frustrated', 'timestamp': '2024-10-01 11:20:00'}
{'user_id': 2, 'task_id': 3, 'time_spent': 18, 'performance': 1, 'emotion': 'neutral', 'timestamp': '2024-10-01 11:40:00'}
{'user_id': 3, 'task_id': 1, 'time_spent': 22, 'performance': 1, 'emotion': 'happy', 'timestamp': '2024-10-01 12:00:00'}
{'user_id': 3, 'task_id': 2, 'time_spent': 12, 'performance': 1, 'emotion': 'excited', 'timestamp': '2024-10-01 12:15:00'}
{'user_id': 3, 'task_id': 3, 'time_spent': 35, 'performance': 0, 'emotion': 'sad', 'timestamp': '2024-10-01 12:30:00'}
{'user_id': 4, 'task_id': 1, 'time_spent': 40, 'performance': 1, 'emotion': 'happy', 'timestamp': '2024-10-01 13:00:00'}
{'user_id': 4, 'task_id': 2, 'time_spent': 5, 'performance': 0, 'emotion': 'frustrated', 'timestamp': '2024-10-01 13:10:00'}
{'user_id': 4, 'task_id': 3, 'time_spent': 30, 'performance': 1, 'emotion': 'neutral', 'timestamp': '2024-10-01 13:20:00'}"""

# Write the log data to a file
with open('user_interactions.log', 'w') as f:
    f.write(log_data)


In [5]:
# Load the log data
import ast

log_entries = []
with open('user_interactions.log', 'r') as f:
    for line in f:
        log_entries.append(ast.literal_eval(line.strip()))  # Convert string representation to a dictionary

# Create a DataFrame from the log entries
logs_df = pd.DataFrame(log_entries)
print(logs_df.head())


   user_id  task_id  time_spent  performance     emotion            timestamp
0        1        1          15            1       happy  2024-10-01 10:00:00
1        1        2          20            1     neutral  2024-10-01 10:15:00
2        1        3          10            0         sad  2024-10-01 10:30:00
3        2        1          30            1       happy  2024-10-01 11:00:00
4        2        2          25            0  frustrated  2024-10-01 11:20:00


In [6]:
# Import necessary libraries
import pandas as pd
import numpy as np
import cv2
import logging
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from deepface import DeepFace
import time

# Set up logging for user interactions
logging.basicConfig(filename='user_interactions.log', level=logging.INFO)

# Function to simulate user interaction logging
def log_interaction(user_id, task_id, time_spent, performance, emotion):
    log_data = {
        'user_id': user_id,
        'task_id': task_id,
        'time_spent': time_spent,
        'performance': performance,
        'emotion': emotion,
        'timestamp': pd.Timestamp.now()
    }
    logging.info(log_data)

# Simulated user interaction function
def simulate_user_interaction():
    user_id = np.random.randint(1, 5)  # Simulate user ID
    task_id = np.random.randint(1, 10)  # Simulate task ID
    time_spent = np.random.randint(5, 60)  # Time spent in seconds
    performance = np.random.randint(0, 2)  # Simulated performance (0 or 1)

    # Simulate emotion detection
    frame = np.zeros((100, 100, 3), dtype=np.uint8)  # Dummy frame for testing
    emotion = detect_emotion(frame)  # Placeholder for emotion detection

    # Log the interaction
    log_interaction(user_id, task_id, time_spent, performance, emotion)

# Function to detect emotion from a frame (using DeepFace)
def detect_emotion(frame):
    try:
        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        return result[0]['dominant_emotion']  # Get the dominant emotion
    except Exception as e:
        return 'unknown'  # Return unknown if detection fails

# Simulate user interactions
for _ in range(10):  # Simulate 10 interactions
    simulate_user_interaction()
    time.sleep(1)  # Wait for a second between interactions

# Load interaction data
# Note: In a real scenario, this data would be collected over time and stored in a database.
data = pd.read_csv('user_interactions.log')

# Data Preprocessing
# Example: Convert logging info to DataFrame
logs_df = pd.DataFrame([eval(line) for line in open('user_interactions.log').readlines()])
print(logs_df.head())

# Feature Engineering
# Create feature and target variables
X = logs_df[['time_spent', 'user_id']]  # Features (you can add more features)
y = logs_df['performance']  # Target variable (performance)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Model Accuracy: {accuracy:.2f}')

# Simulate real-time emotion detection (you can run this cell separately)
cap = cv2.VideoCapture(0)  # Use webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect emotion from the frame
    emotion = detect_emotion(frame)
    cv2.putText(frame, f'Emotion: {emotion}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the video frame with emotion
    cv2.imshow('Webcam Feed', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


   user_id  task_id  time_spent  performance     emotion            timestamp
0        1        1          15            1       happy  2024-10-01 10:00:00
1        1        2          20            1     neutral  2024-10-01 10:15:00
2        1        3          10            0         sad  2024-10-01 10:30:00
3        2        1          30            1       happy  2024-10-01 11:00:00
4        2        2          25            0  frustrated  2024-10-01 11:20:00
Model Accuracy: 0.67
